In [2]:
import torch
import os
import cv2
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as utils
import numpy as np
from torch.autograd import Variable
from torch import optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [12]:
# Define dataset class
class FaceDateSet(Dataset):
    """lfw face data set."""

    def __init__(self, root_dir, split_file, transform = None):
        self.root_dir = root_dir
        self.split_file = split_file
        self.transform = transform
        self.img_paths = self.parse_files()

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        # Get items from path here
        img1_path = os.path.join(self.root_dir, self.img_paths[idx][0])
        img2_path = os.path.join(self.root_dir, self.img_paths[idx][1])
        img_label = map(float,self.img_paths[idx][2])
        img_label = torch.from_numpy(np.array(img_label)).float()
        img1 = Image.open(img1_path)
        img2 = Image.open(img2_path)
        img1 = img1.convert('RGB')
        img2 = img2.convert('RGB')
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        sample = {'img1': img1, 'img2': img2, 'label': img_label}
        return sample

    def parse_files(self):
        img_paths = []
        with open(self.split_file) as f:
            img_paths = f.readlines()
        img_paths = [x.split() for x in img_paths]
        return img_paths

In [4]:
# Define deep neural network
class SiameseNet(nn.Module):

    def __init__(self):
        super(SiameseNet, self).__init__()
        self.nn1 = nn.Sequential(
            nn.Conv2d(3,64,5,padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(64,128,5,padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(128,256,3,padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2,stride=2),
            nn.Conv2d(256,512,3,padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
        )

        self.nn2 = nn.Sequential(
            nn.Linear(131072,1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(1024),
        )

        self.nn3 = nn.Sequential(
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def net_forward(self,x):
        temp = self.nn1(x)
        temp = temp.view(temp.size()[0], -1)
        output = self.nn2(temp)
        return output

    def forward(self,x1,x2):
        output1 = self.net_forward(x1)
        output2 = self.net_forward(x2)
        output12 = torch.cat((output1,output2),1)
        output = self.nn3(output12)
        return output

In [13]:
data_trans = transforms.Compose([transforms.Scale((128,128)),transforms.ToTensor()])
face_train = FaceDateSet(root_dir='lfw', split_file='train.txt',transform = data_trans)
train_loader = DataLoader(face_train, batch_size=8, shuffle=True, num_workers=4)

In [14]:
iterrr = iter(train_loader)
print next(iterrr)
# print face_train[0]

{'img2': 
( 0 , 0 ,.,.) = 
  0.9333  0.9255  0.9216  ...   0.9490  0.9490  0.9451
  0.9137  0.9294  0.9216  ...   0.9490  0.9451  0.9412
  0.8627  0.9333  0.9373  ...   0.9529  0.9490  0.9490
           ...             ⋱             ...          
  0.4784  0.5098  0.5176  ...   0.5255  0.5333  0.5137
  0.3961  0.4549  0.4745  ...   0.5569  0.5333  0.4627
  0.4196  0.4392  0.4745  ...   0.6157  0.5255  0.4431

( 0 , 1 ,.,.) = 
  0.9412  0.9333  0.9294  ...   0.9529  0.9529  0.9490
  0.9216  0.9373  0.9294  ...   0.9529  0.9490  0.9451
  0.8706  0.9412  0.9451  ...   0.9569  0.9529  0.9529
           ...             ⋱             ...          
  0.5098  0.5373  0.5373  ...   0.5412  0.5490  0.5333
  0.4392  0.4941  0.5020  ...   0.5725  0.5490  0.4863
  0.4706  0.4824  0.5059  ...   0.6314  0.5451  0.4667

( 0 , 2 ,.,.) = 
  0.9216  0.9137  0.9098  ...   0.9333  0.9333  0.9294
  0.9020  0.9176  0.9059  ...   0.9333  0.9294  0.9255
  0.8510  0.9216  0.9216  ...   0.9451  0.9333  0.9333
  

In [ ]:
# Training the net
net = SiameseNet()
optimizer = optim.Adam(net.parameters(), lr = 1e-6)
loss_fn = nn.BCELoss()
total_epoch = 30 
for epoch in range(total_epoch):
    for batch_idx, batch_sample in enumerate(train_loader):
        img1 = batch_sample['img1']
        img2 = batch_sample['img2']
        label = batch_sample['label']
        # label = label.view(label.numel(),-1)
        img1, img2, y = Variable(img1), Variable(img2), Variable(label)
        optimizer.zero_grad()
        y_pred = net(img1, img2)
        bce_loss = loss_fn(y_pred, y)
        bce_loss.backward()
        optimizer.step()

        if batch_idx % 50 == 0:
            print "Epoch %d, Batch %d Loss %f" % (epoch, batch_idx, bce_loss.data[0])

Epoch 0, Batch 0 Loss 0.569435


In [8]:
net = SiameseNet()
optimizer = optim.Adam(net.parameters(), lr = 1e-6)
loss_fn = nn.BCELoss()
total_epoch = 2
for epoch in range(total_epoch):
    for batch_idx, batch_sample in enumerate(train_loader):
        img1 = batch_sample['img1']
        img2 = batch_sample['img2']
        label = batch_sample['label'].float()
        label = label.view(label.numel(),-1)
        img1, img2, y = Variable(img1), Variable(img2), Variable(label)
        optimizer.zero_grad()
        y_pred = net(img1, img2)
        bce_loss = loss_fn(y_pred, y)
        bce_loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print "Epoch %d, Batch %d Loss %f" % (epoch, batch_idx, bce_loss.data[0])

Epoch 0, Batch 0 Loss 0.692584


Process Process-9:
Process Process-10:
Process Process-12:
Process Process-11:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 34, i

KeyboardInterrupt: 

In [4]:
x = Variable(torch.ones(2,2),requires_grad=True)
print(x)
y = x+2
print(y)
print(y.grad_fn)
z = y*y*3
out = z.mean()
print(z,out)
out.backward()
print(x.grad)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [4]:
weights_dir = "./model_best.pth.tar"
net = SiameseNet()
net.load_state_dict(torch.load(weights_dir))
loss_fn = nn.BCELoss()

# Testing on the training data
data_trans1 = transforms.Compose([transforms.ToPILImage(),transforms.Scale((128,128)),transforms.ToTensor()])
face_test1 = FaceDateSet(root_dir='lfw', split_file='train.txt', transform = data_trans1)
test1_loader = DataLoader(face_test1, batch_size=1, shuffle=False)
total_loss = 0.0
for batch_idx, batch_sample in enumerate(test1_loader):
    img1 = batch_sample['img1']
    img2 = batch_sample['img2']
    label = batch_sample['label'].float()
    label = label.view(label.numel(),-1)
    img1, img2, y = Variable(img1), Variable(img2), Variable(label)
    y_pred = net(img1, img2)
    bce_loss = loss_fn(y_pred, y)
    if batch_idx % int(len(face_test1)/10) == 0:
        print "Batch %d Loss %f" % (batch_idx, bce_loss.data[0])
    total_loss += bce_loss.data[0]
mean_loss = total_loss / float(len(face_test1))
print "Average BCE loss on training data is: ", mean_loss

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


1.00000e-06 *
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  3.3438  0.0000
  3.2986  0.0000  0.0000
[torch.FloatTensor of size 5x3]



In [6]:
x.add_(x)
print(x)


1.00000e-06 *
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  6.6877  0.0000
  6.5972  0.0000  0.0000
[torch.FloatTensor of size 5x3]



In [9]:
b = x.numpy()
print(b)

[[  5.31873886e-37   0.00000000e+00   5.07241471e-12]
 [  9.16112884e-41   4.90654739e-12   9.16112884e-41]
 [  2.37309200e-37   0.00000000e+00   2.37310276e-37]
 [  0.00000000e+00   6.68766006e-06   9.16112884e-41]
 [  6.59719444e-06   9.16112884e-41   5.06125003e-12]]


In [11]:
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a,1,out=a)
print(a)
print(b)

[ 2.  2.  2.  2.  2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]

